In [ ]:
import os
import pickle
import pandas as pd
import tensorflow as tf
import numpy as np

from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.metrics import log_loss
from tqdm import tqdm
from scipy.sparse import coo_matrix
from utils import load_pickle
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
from utils import load_pickle,dump_pickle

In [ ]:
a = load_pickle('../raw_data/train.pkl')

In [ ]:
b = a.groupby('positionID').size()

In [ ]:
p_index = b[b<5].index

In [ ]:
test[test.positionID.isin(p_index)]

In [ ]:
a.groupby(['clickDaY'])['label'].mean().plot()

In [ ]:
a.groupby(['clickDaY'])['label'].size().plot()

In [ ]:
train_x = pickle.load(open("../cache_pkl/offline_17_27_train_x.pkl",'rb'))
valid_x = pickle.load(open("../cache_pkl/offline_17_27_valid_x.pkl",'rb'))

In [ ]:
COLUMNS = ['age_cut','gender', 'education', 'marriageStatus', 'haveBaby','appID','creativeID']

In [ ]:
feature_names = [ 'age','creativeID', 'userID',
       'positionID', 'connectionType', 'telecomsOperator', 
       'adID', 'camgaignID', 'advertiserID', 'appID', 'appPlatform',
       'appCategory', 'sitesetID', 'positionType',   
       'gender', 'education', 'marriageStatus',
       'haveBaby', 'ht_province',  'rd_province',
                  'clickHour',
       'user_ad_click_day', 'user_camgaign_click_day', 'user_app_click_day',
       'user_site_click_day', 'user_click_day', 'user_ad_click_hour',
       'user_camgaign_click_hour', 'user_app_click_hour',
       'user_site_click_hour',# 'installed_cate_0', 'installed_cate_1',
      # 'installed_cate_2', 'installed_cate_3', 'installed_cate_4',
       #'installed_cate_5'
                ]# 'hometown','residence','time'
      #'score'
cate_features = ['gender', 'education', 'marriageStatus', 'haveBaby','clickHour',
                 'ht_province','rd_province','creativeID','adID', 
        'connectionType', 'telecomsOperator','sitesetID', 'positionType',#'userID',
               'camgaignID','appCategory','advertiserID','appPlatform','appID','positionID', 
              
                #  'app_age_audiences',
              ]# 'time' 


try:
    feature_names.remove('clickTime')
except:
    pass

In [ ]:
all_data = train_x.append(valid_x)

In [ ]:
lbe = LabelEncoder()

In [ ]:
id_size_dict ={}
for feature in cate_features:
    all_data[feature] = lbe.fit_transform(all_data[feature])
    id_size_dict[feature] = all_data[feature].max()+1

In [ ]:
train_x = all_data[all_data.clickDay<27]
valid_x = all_data[all_data.clickDay==27]

In [ ]:
id_size_list = []

In [ ]:
for feature in cate_features:
    id_size_list.append(id_size_dict[feature])

In [ ]:
def run(fetches,sess, X=None, y=None,keep_prob=1.0 ):
        feed_dict = {}
        if type(X) is list:
            for i in range(len(X)):
                feed_dict[id_inputs[i]] = X[i]
                #print(feed_dict[id_inputs[i]])
        else:
            feed_dict[id_inputs] = X
        feed_dict[keep_prob_value] = keep_prob
        if y is not None:
            feed_dict[label_inputs] = y
        return sess.run(fetches, feed_dict)

In [ ]:
input_feature = cate_features#['appID','gender','connectionType']
embedding_size =8
id_dtype= tf.int32
id_field_num = len(input_feature)
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(0)
    global_step =  tf.Variable(0,dtype=tf.int32,trainable=False,name='global_step')

    with tf.name_scope('ID_input'):
        id_inputs =[tf.placeholder(id_dtype,shape=[None,]) for i in range(id_field_num)]
        label_inputs = tf.placeholder(tf.float32)
        keep_prob_value = tf.placeholder(tf.float32,)
    
    with tf.name_scope('Embedding_matrix'):
        embeddings = [tf.Variable(tf.ones([id_size_list[i],embedding_size],)) for i in range(0,id_field_num)]
        #embedding1 = tf.Variable(tf.ones([id_size,embedding_size],))
        #embedding2 = tf.Variable(tf.ones([id_size,embedding_size],))
        #embedding3 = tf.Variable(tf.ones([id_size,embedding_size],))
        onehot_embeddings = [tf.Variable(tf.random_normal([id_size_list[i],1])) for i in range(0,id_field_num)]
        #onehot_embedding1 = tf.Variable(tf.random_normal([id_size,1]))
        #onehot_embedding2 = tf.Variable(tf.random_normal([id_size,1]))
        #onehot_embedding3 = tf.Variable(tf.random_normal([id_size,1]))
    
    with tf.name_scope('Embedding_layer'):
        embeds = [tf.nn.embedding_lookup(embeddings[i],id_inputs[i]) for i in range(0,id_field_num)]
        #embed1 = tf.nn.embedding_lookup(embedding1,appID)
        #embed2 = tf.nn.embedding_lookup(embedding2,genderID)
        #embed3 = tf.nn.embedding_lookup(embedding3,connectionTypeID)
        
        onehot_embeds = [tf.nn.embedding_lookup(onehot_embeddings[i],id_inputs[i]) for i in range(0,id_field_num)]
        #onehot_embed1 = tf.nn.embedding_lookup(onehot_embedding1,appID)
        #onehot_embed2 = tf.nn.embedding_lookup(onehot_embedding2,genderID)
        #onehot_embed3 = tf.nn.embedding_lookup(onehot_embedding3,connectionTypeID)
    with tf.name_scope('Hidden_layer'):
        nn_input = tf.concat(embeds,0)
        fc_1 = tf.contrib.layers.fully_connected(nn_input,200,activation_fn=tf.nn.relu,)
        drop_1 = tf.nn.dropout(fc_1,keep_prob_value)
        nn_logit = tf.contrib.layers.fully_connected(drop_1,1,activation_fn=None)
    with tf.name_scope('FM_layer'):
        fm_units = []
        embed_list=embeds#[embed1,embed2,embed3]
        for i in range(0,id_field_num):
            for j in range(i+1,id_field_num):
                temp = tf.reduce_sum(tf.multiply(embed_list[i],embed_list[j]),axis=1)
                fm_units.append(temp)

        sum_fm_units =tf.reduce_sum(fm_units,0)
        single_onehot_list = tf.concat(onehot_embeds,1)
        sum_linear_units = tf.reduce_sum(single_onehot_list,1)
    with tf.name_scope('Output'):
        
        fm_logits = tf.add(sum_fm_units,sum_linear_units)
        logits = fm_logits #+ nn_logit
        prob = tf.sigmoid(logits)
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=label_inputs,logits=logits))
        va_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=label_inputs,logits=logits))
        optimizer = tf.train.AdamOptimizer().minimize(loss,global_step=global_step)
    with tf.name_scope('Summary'):
        tf.summary.scalar('loss', loss)
        summary_op = tf.summary.merge_all()

In [ ]:
input_feature = cate_features
train_size = train_x.shape[0]
valid_size = valid_x.shape[0]
train_batch_size = 1024#2560#10240
valid_batch_size = 2560#2560
train_batch_num = train_size//train_batch_size
valid_batch_num = valid_size//valid_batch_size
num_round = 100#1000
va_input_X = [valid_x[feature].values for feature in input_feature]
va_input_y = valid_x['label'].values

with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./deepFM_log/',graph=sess.graph)
    sess.run(tf.global_variables_initializer())
    for i in range(num_round):
        for j in tqdm(range(0,1+train_batch_num)): 
            X_i = train_x.iloc[j*train_batch_size:min((j+1)*train_batch_size,train_size)]
            input_X = [X_i[feature].values for feature in input_feature]#X_i[input_feature]
            #print(type(input_X.values))
            input_y =X_i['label'].values
            #em,nn = sess.run([embeds,nn_input],)
            _,l,summary = run([optimizer,loss,summary_op],sess=sess,X = input_X,y = input_y,keep_prob=0.8)
            #logit = run([logits],sess=sess,X = input_X,y = input_y)
            #print(l)
        print(l)
        writer.add_summary(summary, i) 
        va_loss_sum = 0
        for j in range(0,1+valid_batch_num):
            X_i = valid_x.iloc[j*valid_batch_size:min((j+1)*valid_batch_size,valid_size)]
            input_X = [X_i[feature].values for feature in input_feature]#X_i[input_feature]
            input_y =X_i['label'].values
            va_batch_loss = run([va_loss,],sess=sess,X=input_X,y=input_y,keep_prob=1.0)
            #print(va_batch_loss)
            va_loss_sum += va_batch_loss[0]
        print('epoch {0} va-loss {1}'.format(i,va_loss_sum/valid_size))

        saver.save(sess,'save_model',global_step=global_step)       